# Air Liquide coding challenge
## Corey Ducharme

The coding challenge relates to the problem of recommendation systems. My understanding of the prompt is that I had to develop 2 algorithms for user movie recommendation using the movielens dataset. The first algorithm must be either a user-based method or an item-based neighborhood method. The second algorithm must be a matrix factorization collaborative filtering algorithm. Deliverable is the code contained in the following notebook. 

# Importing data

In [ ]:
import requests, zipfile, io
r = requests.get("https://files.grouplens.org/datasets/movielens/ml-latest-small.zip", verify=False)

In [ ]:
movie_zip = zipfile.ZipFile(io.BytesIO(r.content))
movie_zip.namelist()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ratings_df = pd.read_csv(movie_zip.open('ml-latest-small/ratings.csv'))
ratings_df.head(5)

Ratings for movies are done by users and are between 0 and 5.

In [ ]:
movies_df = pd.read_csv(movie_zip.open('ml-latest-small/movies.csv'))
movies_df.head(5)

# Recommender Systems

In [ ]:
# Creating the base class for the recommer system
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

class RecommenderSystem:
    def __init__(self, ratings_df, movies_df):
        self.ratings_df = ratings_df
        self.movies_df = movies_df
        
        # Using a sparse representation to reduce memory usage
        # However this slows downs indexing and other operations done on the dataframe significantly
        user_movie_df = ratings_df.pivot(index = "userId", columns = "movieId", values = "rating").fillna(0)
        self.user_movie_df = user_movie_df.astype(pd.SparseDtype("float", 0))
   
    def recommend_movies(self, userid):
        # Generic recommendation function that will be inherited by each different model
        # Based on the models prediction this filters our the same movies and then 
        # prints them out nicely
        user_row = userid - 1 # UserID starts at 1, not 0
        user_predictions = self.predict(user_row)
        
        # Get the user's movie data for filtering
        user_data = self.ratings_df[self.ratings_df.userId == (userid)]
    
        # Recommend the n highest predicted rating movies that the user hasn't seen yet.
        recommendations = user_predictions[~user_predictions["movieId"].isin(user_data["movieId"])]\
            .sort_values(ascending=False, by="prediction")\
            .head(5)

        return recommendations
    
    def recommend_movies_with_info(self, userid):
        # Add movie info to the recommendations
        # Recommendations are a DataFrame
        return self.recommend_movies(userid).merge(self.movies_df, how="left", left_on = "movieId", right_on = "movieId")
    
    def rmse(self):
        return (((self.all_predictions() - self.user_movie_df)**2).sum().sum()/self.user_movie_df.size)**0.5   
        
    def mae(self):
        return (self.all_predictions() - self.user_movie_df).abs().sum().sum()/self.user_movie_df.size
    
    def k_error(self, k):
        # Return the error for a specified k parameter
        self.fit(k)
        return self.rmse()
        

In [ ]:
# The basis of recommender systems is the user-movie interation matrix "R"
rec = RecommenderSystem(ratings_df, movies_df)
rec.user_movie_df

In [ ]:
rec.user_movie_df.sparse.density

# Matrix factorization collaborative filtering recommender system
## Collaborative filtering
Collaborative filtering is a technique used by recommendation systems to make predictions of about the interest of a user based on collecting preferences from many users. Collaborative filtering creates a forecast for a rating for a product which an active user hasn’t rating yet, by building upon existing ratings of other users who have similar ratings as the active user.

## Matrix factorization
Matrix factorization are an unsupervised class of collaborative filtering algorithm where the user-item interaction matrix is decomposed into a lower dimensionality latent variable space. Matrix factorization models are a type of latent-factor model where users and products are rated on a set of latent variables. The matrix factorization method is well suited for tackling the issue of large sparse interaction matrices. 

### Singular Value Decomposition
The most well known matrix factorization method is the Singular Value Decomposition (SVD). SVD is  a fast and efficient algorithm for identifying latent factors in data. 

$R = U \Sigma V^T$

In its simplest form, SVD decomposes a user-iteraction matrix $R$ into three components: $U$ the user features space matrix, $\Sigma$ the diagonal matrix of singular values (weights), and $V^T$ the items features space matrix. $U$ represents how each “like” a feature and $V^t$ represents how each feature is relevant to each item. 

The SVD algorithm approximates the large user-item matrix into a lower rank by keeping only the k most important underlying features for both user and items. Predictions of SVD are simply the recombined approximated latent variables.

In [ ]:
# Creating our SVD class
from scipy.sparse.linalg import svds

class SVDrec(RecommenderSystem):
    def fit(self, k):
        self.U, sigma, self.Vt = svds(self.user_movie_df, k)
        self.sigma = np.diag(sigma) # Convert sigma back into a diagonal matrix
        
    def all_predictions(self):
        # We don't need to keep the predictions df. Since it is very large.
        all_preds = self.U @ self.sigma @ self.Vt
        all_preds_df = pd.DataFrame(all_preds, columns = self.user_movie_df.columns, index = self.user_movie_df.index)
        return all_preds_df

    def predict(self, user_row):
        # Forecast the ratings for a specific user_row
        # Since we didn't save all the predictions. We recalculate them as required predictions.
        # Since this is matrix multiplication we can go fast by just multiplying the desired row at the begining
        user_predictions = self.U[user_row,:] @ self.sigma @ self.Vt
        user_predictions = pd.DataFrame(user_predictions, self.user_movie_df.columns)\
            .rename(columns={0: "prediction"})\
            .reset_index(inplace=False)
        return user_predictions
    
    def out_of_sample_pred(self, user_row):
        # Fit the model on in-sample data
        self.U, sigma, self.Vt = svds(self.train, k)
        self.sigma = np.diag(sigma) # Convert sigma back into a diagonal matrix
        
        

In [ ]:
svd_rec = SVDrec(ratings_df, movies_df)
svd_rec.fit(50)

In [ ]:
svd_rec.all_predictions()

In [ ]:
(svd_rec.all_predictions() - svd_rec.user_movie_df).abs().sum().sum()

This gives the predictions for all users and for all movies in the dataset. A better forecast would be the top N movies for a specific user that he hasn't seen yet. 

In [ ]:
svd_rec.recommend_movies_with_info(1)

# User-based KNN recommender system

K-NN recommender systems are derived for the nearest neighbors method. They determine the predicted score by looking at what the most similar users have also recommended for the item.

The predicted for an item that a user hasn't seen is the average rating of users which are closest to our user normlized by the distance.

$\hat{r}_{ui} = \frac{\sum sim(u,v) \cdot r_{vi}}{\sum sim(u,v)}$
Here, u is the user, i the item, v other users which are our neighbor and r the recommendation. The sum is done over all k neighbors.

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

class KNNrec(RecommenderSystem):
    def fit(self, k):        
        # Cosine distance is 1 - cosine similarity.
        # We want the similarity for our clustering
        self.distmat = 1 - cosine_distances(self.user_movie_df)
        self.k = k
        # Unfortunately pandas sparse is really slow for indexing which is necessary 
        # to predict with it. Converting it back to dense format.
        # TODO Would need to be looked at later for larger datasets
        if pd.api.types.is_sparse(self.user_movie_df[1]):
            self.user_movie_df = self.user_movie_df.sparse.to_dense()
        
    def nearest_neighbors(self, i):
        # Return the index of the nearest neighbors excluding yourself
        return np.argsort(self.distmat[i])[::-1][1:self.k+1]
    
    def predict_internal(self, i):
        neighbors = self.nearest_neighbors(i)
        k = self.k
        prediction = sum([self.user_movie_df.iloc[id]*self.distmat[k, id] for id in neighbors])\
            /sum([self.distmat[k,id] for id in neighbors])
        return prediction
    
    def predict(self, i):
        user_predictions = pd.DataFrame(self.predict_internal(i))\
            .rename(columns={0: "prediction"})\
            .reset_index(inplace=False)
        return(user_predictions)
    
    def all_predictions(self):
        all_preds = [self.predict_internal(user) for user in np.arange(0,len(self.user_movie_df),1)]
        all_preds = pd.DataFrame(all_preds, index = self.user_movie_df.index)
        return all_preds

In [ ]:
knn_rec = KNNrec(ratings_df, movies_df)

In [ ]:
knn_rec.fit(5)

In [ ]:
knn_rec.all_predictions()

This gives the predictions for all users and for all movies in the dataset. A individual forcast would be the top N movies for a specific user that he hasn't seen yet. 

In [ ]:
knn_rec.recommend_movies_with_info(1)

# Performance comparison

Measuring the performance of recommender systems is a difficult problem in and of itself. Numerous error measurement exists and they vary depending on the class of recommnder systems. Furthermore, error measurements should be tailored to the industrial use case. No metric is better than how real customers interact with recommendations produced in the real world.

In the work above, both methods used can be considered as user-based. They use other users information to predict the ratings for a new user. This allows us to compare them directly on those predictions. 

We chose to use the common RMSE between the predicted ratings and the real ratings.

Choosing an error measurement allows us to optimize the models on a training sample. Both methods have a hyperparameter which we can iterate over to determine the optimal value. 

For SVD, the hyperparameter is the number of features kept in the lower ranks.

For KNN, the hyperparameter is the number of clusters. 

In [ ]:
svd_errors = [svd_rec.k_error(k) for k in np.arange(1,100, 5)]

In [ ]:
knn_error = [knn_rec.k_error(k) for k in np.arange(1,100, 5)]

In [ ]:
import plotly.express as px

fig = px.line(x = np.arange(1,100, 5), y = [svd_errors, knn_error])
fig.data[0].name = "SVD"
fig.data[1].name = "KNN"

fig.update_layout(yaxis_title='RMSE', xaxis_title = 'k')
fig.show()

The in-sample error of the SVD model will only decrease as k increases. A more thorough evaluation of the error is required to evaluate the models, i.e. creating train, validation and test datasets. Which would allow us to determine the best k which does not overfit the data. This same procedure can be repeated for the KNN method. Although one can determine a robust value of k on training data only.

Since both SVD and KNN are user-based, they both suffer from the cold start problem. If a user has no history, he cannot be linked to other users and thus no recommendation can be made. In these cases, I the use of item-based recommendation systems is recommended. KNN can be easily adapted to item-based recommendations. Item-based recommendations will exclusively determine a recommendation for a product based on similarity of the product to others. 

The debate between the superiority of user- vs item- based recommender systems has led to the more modern approach of using a combined multi-criteria recommender system which can combine both user and item based methods when performing recommendations. Furthermore, one can and should include any business specific parameters and needs to the recommender systems being developped.